<a href="https://colab.research.google.com/github/Madhusudan0012/Artificial-intelligence-./blob/main/Hospital_Readmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,SimpleRNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report

In [ ]:
file_path = '/content/hospital_readmissions.csv'
data = pd.read_csv(file_path)

In [ ]:
print(data.head())
print(data.info())

In [4]:
data['readmitted'] = data['readmitted'].map({'Yes': 1, 'No': 0})

In [5]:
data.fillna(0, inplace=True)

In [6]:
label_encoders = {}
for col in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [7]:
X = data.drop('readmitted', axis=1)
y = data['readmitted']

In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
X = X.reshape(X.shape[0], 1, X.shape[1])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9685 - loss: 0.2540 - val_accuracy: 1.0000 - val_loss: 6.7147e-04
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 7.0612e-04 - val_accuracy: 1.0000 - val_loss: 1.1561e-04
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 1.6982e-04 - val_accuracy: 1.0000 - val_loss: 3.9837e-05
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 7.2446e-05 - val_accuracy: 1.0000 - val_loss: 1.8025e-05
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 1.0000 - loss: 3.9281e-05 - val_accuracy: 1.0000 - val_loss: 9.3081e-06
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 1.0000 - loss: 2.1921e-05 - val_accuracy: 1.0000 - val_loss: 5.3143e-06
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 1.0000 - loss: 1.3711e-05 - val_accuracy: 1.0000 - val_loss: 3.1455e-06
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step

In [13]:
y_pred = (model.predict(X_test) > 0.5).astype("int32").flatten()
print(classification_report(y_test, y_pred, labels=[0, 1], target_names=['No', 'Yes']))

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
              precision    recall  f1-score   support

          No       1.00      1.00      1.00      5000
         Yes       0.00      0.00      0.00         0

    accuracy                           1.00      5000
   macro avg       0.50      0.50      0.50      5000
weighted avg       1.00      1.00      1.00      5000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [ ]:
def predict_readmission(input_data):
    input_data = scaler.transform(np.array(input_data).reshape(1, -1))
    input_data = input_data.reshape(1, 1, input_data.shape[1])
    prediction = model.predict(input_data)
    return 'Yes' if prediction > 0.5 else 'No'

In [ ]:
#example of this model
sample_patient = X_test[0].reshape(1, 1, X_test.shape[2])
print("Prediction:", predict_readmission(sample_patient.reshape(1, -1)))


In [ ]:
new_patient_data = [45, 1, 120, 80, 150, 0, 2, 1, 1, 3, 0, 1,1,1,1,1]


print("Prediction:", predict_readmission(new_patient_data))


In [22]:
model.save('readmission_prediction_model.h5')

In [27]:
pip install streamlit pyngrok


In [30]:
%%writefile app.py
import streamlit as st
import pickle
import numpy as np

# Load Model
def load_model():
    with open("model.pkl", "rb") as file:
        model = pickle.load(file)
    return model

# Ensure model.pkl exists
try:
    model = load_model()
except FileNotFoundError:
    st.error("Model file 'model.pkl' not found! Please upload it.")

# Streamlit UI
st.title("ML Model Deployment with Streamlit on Google Colab")
user_input = st.text_input("Enter a value:")
if st.button("Predict"):
    try:
        prediction = model.predict([[float(user_input)]])  # Example prediction
        st.write(f"Prediction: {prediction[0]}")
    except:
        st.error("Error making prediction. Ensure the model is loaded properly.")


Writing app.py


In [32]:
model.save("name.h5")

In [33]:
from google.colab import files
uploaded = files.upload()  # Manually upload 'model.pkl'


Saving name.h5 to name (1).h5


In [35]:
!wget -q -O - ipv4.icanhazip.com

35.245.187.10


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.187.10:8501

⠸⠼⠴your url is: https://modern-kings-sleep.loca.lt
